## First implementation of the code.

We are trying to follow the meassurements done by [1]. The first try was to implement the following algorithm:

Starting with state |1 > to perform the measurement.

1. Initialize all qubits at state |1>
2. Wait 1 µs.
3. Measure the state of all qubits. 

Repeat the process each 100 µs.

And for that we started by implementing the following code (v0):

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
#from qmiotools.integrations.qiskitqmio import FakeQmio

# number of qubits
num_qubits = 5

#backend = FakeQmio()
backend = AerSimulator() #qmiotools not installed locally

def create_circuit(num_qubits):

    # we are measuring 32 qubits so we need 32 classic qubits to save the results
    qc = QuantumCircuit(num_qubits, num_qubits)
    # initialice all qubits to state |1⟩
    qc.x(range(num_qubits))
    
    qc.delay(1, range(num_qubits), unit='us') # delay of 1 µs
    # measuring all qubits
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

qc = create_circuit(num_qubits)

# repeating each 100 µs for 100s
# num_repetitions = int(100 / 0.0001)
num_repetitions = 10

for i in range(num_repetitions):

    qc.delay(100, range(num_qubits), unit='us')  # delay of  100 µs

    # initializing again the circuit ?
    qc = create_circuit(num_qubits)
    
    # executing the circuit ? 
    qct = transpile(qc, backend)
    job = backend.run(qct, shots=1024)
    result = job.result()
    counts = result.get_counts(qc) # qc ? 
    
    # printing results
    print(f"Repetition {i+1} - FakeQmio Counts:", counts)

Repetition 1 - FakeQmio Counts: {'11111': 1024}
Repetition 2 - FakeQmio Counts: {'11111': 1024}
Repetition 3 - FakeQmio Counts: {'11111': 1024}
Repetition 4 - FakeQmio Counts: {'11111': 1024}
Repetition 5 - FakeQmio Counts: {'11111': 1024}
Repetition 6 - FakeQmio Counts: {'11111': 1024}
Repetition 7 - FakeQmio Counts: {'11111': 1024}
Repetition 8 - FakeQmio Counts: {'11111': 1024}
Repetition 9 - FakeQmio Counts: {'11111': 1024}
Repetition 10 - FakeQmio Counts: {'11111': 1024}


Which in fact works but takes a long time to execute. We were facing 1h30min for the code with 10 qubits and 10 repetitions...

After debating with Andrés Gómez from CESGA there are some characteristics about the QPU we haven't taken into account: 

- The coherence times of the qubits are longer than what we expected.They are about $T_1 = 60-120 \mu s$. If we want the state to be again at |0> we need to wait $3-5 T_1$. 
- In order for the system to keep an exact period between measurements we need to use the *repetition_period* argument in the run command.
  

### Bibliografy.

[1] Matt McEwen et all. Resolving catastrophic error bursts from cosmic rays in large arrays of superconducting qubits. Nature Physics, 18(1):107–111, 2022.